In [130]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import math

In [131]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.precision', 4)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [132]:
path = '/Users/admin/fin427/project1/'

# Import data and create a quarter variable for alignment with quarterly financial information
returns = pd.read_excel(path + 'Excel03 Data 20230128.xlsx', sheet_name='ret06')
returns1 = returns
returns1['quarter'] = returns1['month'] + pd.offsets.QuarterEnd(0)
print(returns1.head(6))
print(returns1.columns)

   PERMNO       DATE     CUSIP                        COMNAM TICKER  PERMCO  \
0   50906 1995-01-31  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   
1   50906 1995-02-28  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   
2   50906 1995-03-31  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   
3   50906 1995-04-28  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   
4   50906 1995-05-31  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   
5   50906 1995-06-30  00088630  A D C TELECOMMUNICATIONS INC   ADCT    2902   

   SHRCD     VOL      RET     RETX  ...  d_4802_Finance_NEC  d_4803_Insurance  \
0     11   34941 -0.02000 -0.02000  ...                   0                 0   
1     11   60361  0.13776  0.13776  ...                   0                 0   
2     11   87371  0.05830  0.05830  ...                   0                 0   
3     11   78824  0.11864  0.11864  ...                   0                 0   
4     11   95578 -0.06061 -0.06061  ...  

In [133]:
sales = pd.read_excel(path + 'SP400_Sales_20230131.xlsx', sheet_name='Sales')
sales1 = sales
sales1 = sales1.dropna()
sales1 = sales1[sales1['Sales/Turnover (Net)'] > 0]
sales1['sales'] = sales1['Sales/Turnover (Net)']
sales1['quarter'] = sales1['Data Date'] + pd.offsets.QuarterEnd(0)
sales1['leadquarter'] = sales1['quarter'] + pd.offsets.QuarterEnd(1)
sales1['lnsales'] = np.log(sales1['sales']*1000)
univ_sales = sales1.sales.describe()
univ_lnsales = sales1.lnsales.describe()
sales1['cusip8'] = sales1['CUSIP'].str[0:8]
print(univ_sales)
print(univ_lnsales)
print(sales1.head(50))
print(sales1.columns)
print(sales1)

count   108646.00000
mean      1994.09869
std       5774.02505
min          0.00100
25%        265.24175
50%        733.04850
75%       1889.35075
max     263966.00000
Name: sales, dtype: float64
count   108646.00000
mean        13.40562
std          1.59087
min          0.00000
25%         12.48840
50%         13.50497
75%         14.45174
max         19.39133
Name: lnsales, dtype: float64
    Global Company Key  Data Date  Fiscal Year  Fiscal Quarter  \
0                 1013 1994-01-31         1994         1.00000   
4                 1013 1995-01-31         1995         1.00000   
8                 1013 1996-01-31         1996         1.00000   
9                 1013 1996-04-30         1996         2.00000   
10                1013 1996-07-31         1996         3.00000   
11                1013 1996-10-31         1996         4.00000   
12                1013 1997-01-31         1997         1.00000   
13                1013 1997-04-30         1997         2.00000   
14          

In [134]:
#Create RSI Characteristic 
#rollingAmt = 15
#returns1['Up'] = np.where(returns1['RET'] >= 0, 1, 0)
#returns1['Down'] = np.where(returns1['RET'] < 0, 1, 0)
#returns1["Up"] = pd.to_numeric(returns1["Up"])
#returns1['UpDownRatio'] = returns1['Up'].rolling(rollingAmt).mean() / returns1['Down'].rolling(rollingAmt).mean()
#returns1['RSI'] = 100 - 100 / ( 1 + returns1['UpDownRatio'] )
#returns1["RSI"] = returns1["RSI"].fillna(50)
#returns1['RSI'] = returns1['RSI'].shift(-1)
#returns1 = returns1[:-1]
#returns1 = returns1.reset_index(drop=True)

In [135]:
#Create RSI for past days
dailyReturns = pd.read_csv(path + 'dailyPrices.csv')
dailyReturns['RET'] = pd.to_numeric(dailyReturns['RET'], errors='coerce')
dailyReturns['Up'] = np.where(dailyReturns['RET'] >= 0, 1, 0)
dailyReturns['Down'] = np.where(dailyReturns['RET'] < 0, 1, 0)
dailyReturns['UpDownRatio'] = dailyReturns['Up'].rolling(rollingAmt).mean() / dailyReturns['Down'].rolling(rollingAmt).mean()
dailyReturns['RSI'] = 100 - 100 / ( 1 + dailyReturns['UpDownRatio'] )
dailyReturns["RSI"] = dailyReturns["RSI"].fillna(50)
dailyReturns['RSI'] = dailyReturns['RSI'].shift(-2)
dailyReturns = dailyReturns[:-1]
dailyReturns = dailyReturns.reset_index(drop=True)
# Now set the RSI for the quarters


/var/folders/36/nqq61ypd1tvg6r10cs7jqzbm0000gn/T/ipykernel_48256/2385437696.py:2: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  dailyReturns = pd.read_csv(path + 'dailyPrices.csv')


In [136]:
dailyReturns['Date']= pd.to_datetime(dailyReturns['adj-date'])


In [137]:
#Combine 
bigOne = returns1.merge(dailyReturns, how='left', left_on=['CUSIP', 'DATE'], right_on=['CUSIP', 'Date'])


In [138]:
bigOne.dropna(inplace = True)

In [139]:
bigOne.isna().sum()

PERMNO_x                         0
DATE_x                           0
CUSIP                            0
COMNAM_x                         0
TICKER_x                         0
PERMCO                           0
SHRCD                            0
VOL                              0
RET_x                            0
RETX                             0
SHROUT                           0
VWRETD                           0
VWRETX                           0
price                            0
month                            0
mc                               0
lnmc                             0
lag1cusip                        0
lag1month                        0
lag1mc                           0
lag1lnmc                         0
bmret                            0
abret                            0
ISIN                             0
cusip9                           0
FSESNum                          0
FSISNum                          0
FSESName                         0
FSISName            

In [140]:
combined1 = bigOne.merge(sales1, how='left', left_on=['CUSIP', 'quarter'], right_on=['cusip8', 'leadquarter'])
combined1.rename(columns={'sales': 'lag1sales', 'lnsales': 'lag1lnsales'}, inplace=True)
print(combined1.head())
print(combined1.columns)
print(len(returns1.index))
print(len(sales1.index))
print(len(combined1.index))
print(combined1.lag1sales.describe(percentiles=[0.125, 0.875]))
print(combined1.lag1mc.describe(percentiles=[0.125, 0.875]))
print(combined1.lag1lnsales.describe(percentiles=[0.125, 0.875]))
print(combined1.lag1lnmc.describe(percentiles=[0.125, 0.875]))
print(combined1.retadj.describe(percentiles=[0.125, 0.875]))
print(combined1.bmret.describe(percentiles=[0.125, 0.875]))
print(combined1.abretadj.describe(percentiles=[0.125, 0.875]))

# Compute the correlation between lnsales and lag1lnmc
dfcorr = combined1[['lag1lnsales', 'lag1lnmc']].copy()
dfcorr.corr(method='pearson')

   PERMNO_x     DATE_x   CUSIP_x                      COMNAM_x TICKER_x  \
0     50906 1995-01-31  00088630  A D C TELECOMMUNICATIONS INC     ADCT   
1     50906 1995-02-28  00088630  A D C TELECOMMUNICATIONS INC     ADCT   
2     50906 1995-03-31  00088630  A D C TELECOMMUNICATIONS INC     ADCT   
3     50906 1995-04-28  00088630  A D C TELECOMMUNICATIONS INC     ADCT   
4     50906 1995-05-31  00088630  A D C TELECOMMUNICATIONS INC     ADCT   

   PERMCO  SHRCD    VOL    RET_x     RETX  ...  ISO Currency Code  \
0    2902     11  34941 -0.02000 -0.02000  ...                NaN   
1    2902     11  60361  0.13776  0.13776  ...                NaN   
2    2902     11  87371  0.05830  0.05830  ...                NaN   
3    2902     11  78824  0.11864  0.11864  ...                USD   
4    2902     11  95578 -0.06061 -0.06061  ...                USD   

   Calendar Data Year and Quarter  Fiscal Data Year and Quarter  \
0                             NaN                           NaN   


,lag1lnsales,lag1lnmc
lag1lnsales,1.00000,0.63072
lag1lnmc,0.63072,1.00000


In [141]:
combined1.dropna(inplace = True)
for col in combined1.columns:
    print(col)
combined1.isna().sum()

PERMNO_x
DATE_x
CUSIP_x
COMNAM_x
TICKER_x
PERMCO
SHRCD
VOL
RET_x
RETX
SHROUT
VWRETD
VWRETX
price
month
mc
lnmc
lag1cusip
lag1month
lag1mc
lag1lnmc
bmret
abret
ISIN
cusip9
FSESNum
FSISNum
FSESName
FSISName
MLIC
MLIL
Group
d
name
d_1100_Non_Energy_Minerals
d_1200_Producer_Manufacturing
d_1300_Electronic_Technology
d_1400_Consumer_Durables
d_2100_Energy_Minerals
d_2200_Process_Industries
d_2300_Health_Technology
d_2400_Consumer_Non_Durables
d_3100_Industrial_Services
d_3200_Commercial_Services
d_3250_Distribution_Services
d_3300_Technology_Services
d_3350_Health_Services
d_3400_Consumer_Services
d_3500_Retail_Trade
d_4600_Transportation
d_4700_Utilities
d_4801_Banks
d_4802_Finance_NEC
d_4803_Insurance
d_4885_Real_Estate_Dev
d_4890_REIT
d_4900_Communications
d_6000_Miscellaneous
numrets
retadj
abretadj
quarter_x
Unnamed: 0.4
Unnamed: 0.3
Unnamed: 0.2
Unnamed: 0.1
Unnamed: 0
PERMNO_y
date
TICKER_y
COMNAM_y
PRC
RET_y
adj-date
DATE_y
Up
Down
UpDownRatio
RSI
Date
Global Company Key
Data Date
F

PERMNO_x                                               0
DATE_x                                                 0
CUSIP_x                                                0
COMNAM_x                                               0
TICKER_x                                               0
PERMCO                                                 0
SHRCD                                                  0
VOL                                                    0
RET_x                                                  0
RETX                                                   0
SHROUT                                                 0
VWRETD                                                 0
VWRETX                                                 0
price                                                  0
month                                                  0
mc                                                     0
lnmc                                                   0
lag1cusip                      

In [145]:
y1 = combined1['abretadj']
x = combined1[['lag1lnmc', 'lag1lnsales', 'RSI',
    'd_1100_Non_Energy_Minerals',
    'd_1200_Producer_Manufacturing',
    'd_1300_Electronic_Technology',
    'd_1400_Consumer_Durables',
    'd_2100_Energy_Minerals',
    'd_2200_Process_Industries',
    'd_2300_Health_Technology',
    'd_2400_Consumer_Non_Durables',
    'd_3100_Industrial_Services',
    'd_3200_Commercial_Services',
    'd_3250_Distribution_Services',
    'd_3300_Technology_Services',
    'd_3350_Health_Services',
    'd_3400_Consumer_Services',
    'd_3500_Retail_Trade',
    'd_4600_Transportation',
    'd_4700_Utilities',
    'd_4801_Banks',
    'd_4802_Finance_NEC',
    'd_4803_Insurance',
    'd_4885_Real_Estate_Dev',
    'd_4890_REIT',
    'd_4900_Communications']]
x1 = StandardScaler().fit_transform(x)
x = sm.add_constant(x)
x1 = sm.add_constant(x1)
model = sm.OLS(y1, x).fit()
# model = sm.OLS(y1, x1).fit()
print_model = model.summary()
ols_coef = model.params
ols_rsq  = model.rsquared
print(print_model)
print(f'R-squared: {model.rsquared:.4f}')
print(ols_coef)

                            OLS Regression Results                            
Dep. Variable:               abretadj   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     1405.
Date:                Wed, 15 Feb 2023   Prob (F-statistic):               0.00
Time:                        22:32:52   Log-Likelihood:             2.3467e+05
No. Observations:              287763   AIC:                        -4.693e+05
Df Residuals:                  287736   BIC:                        -4.690e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

In [143]:
y1 = combined1['abretadj']
x = combined1[['lag1lnmc', 'lag1lnsales', 'RSI',
    'd_1100_Non_Energy_Minerals',
    'd_1200_Producer_Manufacturing',
    'd_1300_Electronic_Technology',
    'd_1400_Consumer_Durables',
    'd_2100_Energy_Minerals',
    'd_2200_Process_Industries',
    'd_2300_Health_Technology',
    'd_2400_Consumer_Non_Durables',
    'd_3100_Industrial_Services',
    'd_3200_Commercial_Services',
    'd_3250_Distribution_Services',
    'd_3300_Technology_Services',
    'd_3350_Health_Services',
    'd_3400_Consumer_Services',
    'd_3500_Retail_Trade',
    'd_4600_Transportation',
    'd_4700_Utilities',
    'd_4801_Banks',
    'd_4802_Finance_NEC',
    'd_4803_Insurance',
    'd_4885_Real_Estate_Dev',
    'd_4890_REIT',
    'd_4900_Communications']]
x1 = StandardScaler().fit_transform(x)
x1 = sm.add_constant(x1)
lasso = Lasso(alpha = 0.001)
lasso.fit(x1, y1)
lasso_coef = lasso.fit(x1, y1).coef_
lasso_score = lasso.score(x1, y1)
print(f'Lasso score: {lasso_score: .4f}')
print(lasso.intercept_)
print(lasso_coef)
print(x.columns)

Lasso score:  0.1117
0.004390298327933042
[ 0.         -0.00294434  0.          0.03657791  0.         -0.
  0.00077043  0.          0.         -0.          0.00059862 -0.
  0.         -0.         -0.          0.00038018  0.         -0.
  0.         -0.         -0.00145513 -0.00020425  0.         -0.
 -0.         -0.00109283 -0.        ]
Index(['lag1lnmc', 'lag1lnsales', 'RSI', 'd_1100_Non_Energy_Minerals',
       'd_1200_Producer_Manufacturing', 'd_1300_Electronic_Technology',
       'd_1400_Consumer_Durables', 'd_2100_Energy_Minerals',
       'd_2200_Process_Industries', 'd_2300_Health_Technology',
       'd_2400_Consumer_Non_Durables', 'd_3100_Industrial_Services',
       'd_3200_Commercial_Services', 'd_3250_Distribution_Services',
       'd_3300_Technology_Services', 'd_3350_Health_Services',
       'd_3400_Consumer_Services', 'd_3500_Retail_Trade',
       'd_4600_Transportation', 'd_4700_Utilities', 'd_4801_Banks',
       'd_4802_Finance_NEC', 'd_4803_Insurance', 'd_4885_Real_Est

In [144]:
y1 = combined1['abretadj']
x = combined1[['lag1lnmc','lag1lnsales', 'RSI', 
   'd_1100_Non_Energy_Minerals',
    'd_1200_Producer_Manufacturing',
    'd_1300_Electronic_Technology',
    'd_1400_Consumer_Durables',
    'd_2100_Energy_Minerals',
    'd_2200_Process_Industries',
    'd_2300_Health_Technology',
    'd_2400_Consumer_Non_Durables',
    'd_3100_Industrial_Services',
    'd_3200_Commercial_Services',
    'd_3250_Distribution_Services',
    'd_3300_Technology_Services',
    'd_3350_Health_Services',
    'd_3400_Consumer_Services',
    'd_3500_Retail_Trade',
    'd_4600_Transportation',
    'd_4700_Utilities',
    'd_4801_Banks',
    'd_4802_Finance_NEC',
    'd_4803_Insurance',
    'd_4885_Real_Estate_Dev',
    'd_4890_REIT',
    'd_4900_Communications']]
x1 = StandardScaler().fit_transform(x)
x = sm.add_constant(x)
x1 = sm.add_constant(x1)
model = sm.OLS(y1, x1).fit()
# model = sm.OLS(y1, x1).fit()
print_model = model.summary()
ols_coef = model.params
ols_rsq  = model.rsquared
print(print_model)
print(f'R-squared: {model.rsquared:.4f}')
print(ols_coef)

                            OLS Regression Results                            
Dep. Variable:               abretadj   R-squared:                       0.113
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     1405.
Date:                Wed, 15 Feb 2023   Prob (F-statistic):               0.00
Time:                        22:31:50   Log-Likelihood:             2.3467e+05
No. Observations:              287763   AIC:                        -4.693e+05
Df Residuals:                  287736   BIC:                        -4.690e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0044      0.000     21.998      0.0

In [147]:
bigOne.to_csv(path + "RSI_results.csv")